# Notes about decorators

Source: TUTORIAL / Geir Arne Hjelle / Introduction to Decorators: Power UP Your Python Code  
https://www.youtube.com/watch?v=VWZAh1QrqRE  
PyCon US, 2021  

## Using decorators breaks the wrapped function name

We create a decorator called `register` that registers the wrapped function to a dictionary so that it can be called from there. The decoration is performed when the function is defined, and not when the function is subsequently called.

In [1]:
import random
FUNCTIONS = {}

def register(wrapped):
    '''
    Does some processing when the decorated function is defined, and on every subsequent
    call not changes are made, as the function is not changed at all.

    '''

    print(f"Registering {wrapped.__name__}")
    FUNCTIONS[wrapped.__name__] = wrapped
    return wrapped

In [2]:
@register
def roll_dice():
    return random.randint(1, 6)

Registering roll_dice


In [3]:
FUNCTIONS

{'roll_dice': <function __main__.roll_dice()>}

In [4]:
FUNCTIONS["roll_dice"]()

1

In [5]:
roll_dice

<function __main__.roll_dice()>

However, things start to break when we wish to store the names of functions that have already been wrapped in a decorator. Let us build another decorator called `do_twice` that runs a function twice and returns a 2-tuple of the returned values. 

In [6]:
def do_twice(wrapped):
    def wrapper(*args, **kwargs):
        return wrapped(*args, **kwargs), wrapped(*args, **kwargs)
    return wrapper


In [7]:
@do_twice
def roll_dice():
    return random.randint(1, 6)

In [8]:
roll_dice()

(5, 5)

In [10]:
roll_dice

<function __main__.do_twice.<locals>.wrapper(*args, **kwargs)>

Here, we can see that the name of the original source function `roll_dice` has been replaced with the wrapper name `wrapper` from the `do_twice` decorator. To fix this, we can slap on decorators from the standard library that decorate the wrapper function to look like the wrapped function.

In [11]:
import functools

In [15]:
def do_twice(wrapped):
    @functools.wraps(wrapped)
    def wrapper(*args, **kwargs):
        return wrapped(*args, **kwargs), wrapped(*args, **kwargs)
    return wrapper

In [16]:
@do_twice
def roll_dice():
    return random.randint(1, 6)

In [17]:
roll_dice()

(4, 1)

In [18]:
roll_dice

<function __main__.roll_dice()>

We can use this decorator to call the undecorated function as well.

In [20]:
roll_dice.__wrapped__()

5

The `__wrapped__` method is added by `functools.wraps` to the wrapped function to store the original unmodified method.